In [17]:
%pip install llama-index-llms-gemini llama-index
%pip install requests
%pip install gradio
%pip install python-dotenv

In [18]:
import os
import requests
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage
import gradio as gr

In [19]:
# Importa .env do google drive pessoal, trazendo as chaves necessárias

from google.colab import drive
from dotenv import load_dotenv
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
# Chaves importadas do drive
GEMINI_KEY=os.getenv("GEMINI_KEY")
OPENWEATHER_API_KEY=os.environ["OPENWEATHER_API_KEY"]

In [21]:
# Entrada de chaves manual para teste
# GEMINI_KEY="GEMINI_KEY"
# OPENWEATHER_API_KEY="OPENWEATHER_API_KEY"

In [22]:
# Inicia a llm
llm = Gemini(
    model="models/gemini-2.5-flash",
    api_key=GEMINI_KEY
)

/tmp/ipython-input-1171370132.py:2: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(


In [23]:
# Buscar clima
def buscar_clima(cidade: str, lang="pt") -> str:
  try:
    url = f"http://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={OPENWEATHER_API_KEY}&units=metric&lang={lang}"
    resposta = requests.get(url)
    data = resposta.json()

    if data.get("cod") != 200:
        return f"Não consegui encontrar o clima para {cidade}."

    # Obter dados do clima
    desc = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    feels = data["main"]["feels_like"]
    pais = data["sys"]["country"]

    # obter latitude e longitude
    lat = data["coord"]["lat"]
    lon = data["coord"]["lon"]

    # Chamada reversa para obter o estado
    geo_url = f"http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit=1&appid={OPENWEATHER_API_KEY}"
    geo_data = requests.get(geo_url).json()
    estado = geo_data[0].get("state", "Desconhecido")

    # Retorna estas informações para a llm interpretar
    return (
        f"O clima em {cidade} ({estado}, {pais}) é {desc}, "
        f"com temperatura de {temp}°C (sensação térmica de {feels}°C)."
    )
  except Exception as e:
        return f"Erro ao obter o clima: {e}"


In [24]:
# Função que cria o comportamento do bot
def chat_bot(user_input, historico):
  cidade = user_input.strip()
  clima = buscar_clima(cidade)

  # Prompt de instrução de comportamento para o bot
  mensagens = [
      ChatMessage(
          role="system",
          content= f"O usuário pediu o clima no seu estado na resposta: {user_input}. "
                   f"A informação do clima é: {clima}. Responda de forma natural e educada, "
                   f"citando todas as informações obtidas do clima. Após isso recomende ao "
                   f"usuário um passeio em uma das mais importantes cidades deste estado, "
                   f"baseando-se nas informações obtidas do clima.")
      ]

  resposta = llm.chat(mensagens)
  resposta_bot = resposta.message.content
  return resposta_bot

In [25]:
# Criação da interface com Gradio
def criar_interface():
  chat = gr.ChatInterface(
      fn=chat_bot,
      title="Clima Tour Bot🌤️",
      description="Envie ao bot o nome do Estado que você deseja saber sobre o clima e possíveis recomendações!",
      theme="soft"
  )

  chat.launch(debug=True)

In [26]:
criar_interface()

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2654dee2537b6eadd7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2654dee2537b6eadd7.gradio.live
